In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import  nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
import cv2
from torchvision import transforms, utils
import skimage.io as skio



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
_metadata = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")
_metadata.loc[_metadata["Finding Labels"].str.contains("No Finding")]

In [ ]:
!pip install einops

In [ ]:
import einops
def show_image(image):
    plt.figure()
    plt.imshow(image)
    plt.show()

def rearrange_tensor(img):
    return einops.rearrange(img, "c w h -> w h c")

def unnormalize_tensor(img):   
    img = img.detach().numpy()
#     img = einops.rearrange(img, "b c w h -> w h (b c)")
    img = einops.rearrange(img, "c w h -> w h c")

    mean = np.array([0.485, 0.456, 0.406])
    std =  np.array([0.229, 0.224, 0.225])
    img = (img * std) + mean
    return img

def unnormalize_img(img):   
    mean = np.array([0.485, 0.456, 0.406])
    std =  np.array([0.229, 0.224, 0.225])
    img = (img * std) + mean
    return img


def normalize_image(img):
    # normalize all images, this is necessary prepreocessing of inputs for vgg network
    normalize = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize(256), 
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize((0.485, 0.456, 0.406), 
                                          (0.229, 0.224, 0.225))
                    ])
    img = normalize(img)
#     img = img[:3,:,:].unsqueeze(0)
    return img

In [ ]:
class ChestDataset(Dataset):
    def __init__(self, class_name = "Pneumonia", val = False):
        _metadata = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")
        _metadata = _metadata.sample(frac = 1)
        metadata_true = _metadata.loc[_metadata["Finding Labels"].str.contains(class_name)]
        metadata_false = _metadata.loc[_metadata["Finding Labels"].str.contains("No Finding")]
        num_train = int(.6 * len(metadata_true))
        num_val = int(.4 * len(metadata_true))
        train_metadata = pd.concat([metadata_true[:num_train], metadata_false[:num_train]])
        val_metadata = pd.concat([metadata_true[num_train:], metadata_false[num_train : num_train + num_val]])
        self.metadata = pd.concat([train_metadata, val_metadata])
        self.num_train = num_train
        self.num_val = num_val
        self.class_name = class_name
        self.data = {}
        self.val = val
    def __len__(self): 
        if not self.val:
            return self.num_train * 2
        return self.num_val * 2
    def __getitem__(self, idx):
        if self.val:
            idx += self.num_train * 2
        file_name = self.metadata.iloc[idx]["Image Index"]
        row = self.metadata.iloc[idx].name
        folder_num = 1
        if row >= 4999:
            folder_num = (row - 4999) // 10000 + 2
        image_file_path = "/kaggle/input/data/images_" + str(folder_num).zfill(3) + "/images/" + file_name
        img = skio.imread(image_file_path)
        if len(img.shape) >= 3:
            img = img[:, :, 0]
        label = self.metadata.iloc[idx]["Finding Labels"]
#         print (file_name, self.metadata.iloc[idx].name, label)

        # turn 1-channel image to 3-channel image
        img = np.stack((img,)*3, axis=-1)
        # crop & normalize
        img = normalize_image(img)
        
        if self.class_name in label:
#             self.data[idx] = (img, 1)
            return img, 1
#         self.data[idx] = (img, 0)
        return img, 0

In [ ]:
# num_train = 5000
# _metadata = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")
# metadata_true = _metadata.loc[_metadata["Finding Labels"].str.contains("Effusion")].head(num_train//2)
# num_train = min(num_train, len(metadata_true))
# metadata_false = _metadata.loc[~_metadata["Finding Labels"].str.contains("Effusion")].head(num_train//2)
# metadata = pd.concat([metadata_true, metadata_false])
# # len(metadata.loc[_metadata["Finding Labels"].str.contains("Effusion")].head(5000//2))

# val = ChestDataset(class_name = "Pneumonia", val = True)
# count = 0
# for i in tqdm(range(len(val))):
#     if val[i][1] == 1:
#         count += 1
# count

In [ ]:
_dataset = ChestDataset(class_name = "Cardiomegaly")
# from random import sample
# for i in sample(range(len(_dataset)), 10):
#     img, label = _dataset[i]
#     print (label)
#     plt.imshow(img, cmap = "gray")
#     plt.show()

# one_channel_img = _dataset[0][0] 
# rgb_img = np.stack((one_channel_img,)*3, axis=-1)
# show_image(rgb_img)
# transformed_image = normalize_image(rgb_img)
# show_image(rearrange_tensor(transformed_image))
# show_image(unnormalize_tensor(transformed_image))

In [ ]:
batch_size = 16
train_dataset = ChestDataset(class_name = "Cardiomegaly")
train_data_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
validation_dataset = ChestDataset(class_name ="Cardiomegaly", val = True)
validation_data_loader = DataLoader(validation_dataset, batch_size = batch_size, shuffle = True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# !pip -q install vit_pytorch linformer

In [ ]:
# from vit_pytorch.efficient import ViT
# from linformer import Linformer
# efficient_transformer = Linformer(
#     dim=128,
#     seq_len=1025,  # 7x7 patches + 1 cls-token
#     depth=12,
#     heads=8,
#     k=64
# )

# model = ViT(
#     dim=128,
#     image_size=1024,
#     patch_size=16,
#     num_classes=2,
#     transformer=efficient_transformer,
#     channels=1,
# ).to(device)

# using pretrained resnet101 due to results found in https://www.nature.com/articles/s41598-020-74164-z
import torchvision.models as models
model = models.resnet101(pretrained=True)
# model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model = model.to(device)
fc_layers = nn.Linear(in_features=1000, out_features=2,bias=True)
fc_layers = fc_layers.to(device)

In [ ]:
# epochs = 25
# lr = 3e-5
# gamma = 0.7

epochs = 6
lr = 3e-4
gamma = 0.1

In [ ]:
# loss function
# triplet and siamese losses are supposed to be used for typical CBIR tasks
criterion = nn.CrossEntropyLoss() # nn.TripletMarginLoss(margin=0.1) 
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
import tqdm
checkpoint_file = "model_weights"
epoch = 0
# if os.path.exists(checkpoint_file):
#     print('gathering info from checkpoint file')
#     checkpoint = torch.load(checkpoint_file)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     criterion.load_state_dict(checkpoint['optimizer_state_dict'])
#     epoch = checkpoint['epoch'] + 1

for epoch in range(epoch, epochs + epoch):
    epoch_loss = 0
    epoch_accuracy = 0
    
    # Wrap in a progress bar to display progress during training.
    progress_bar = tqdm.tqdm(train_data_loader)
    
    for i, inputs in enumerate(progress_bar):
        data, label = inputs
        data = data.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        output = fc_layers(model(data))
        
        loss = criterion(output, label)
        
        loss.backward()
        optimizer.step()
        progress_bar.set_description(f"Loss: {loss}")
        
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_data_loader)
        epoch_loss += loss / len(train_data_loader)
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in tqdm.tqdm(validation_data_loader):
            data = data.to(device)
            label = label.to(device)

            val_output = fc_layers(model(data))
            val_loss = criterion(val_output, label)
            

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(validation_data_loader)
            epoch_val_loss += val_loss / len(validation_data_loader)
    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': criterion.state_dict(),
                'acc': epoch_accuracy,
                'loss': epoch_loss,
                'validation_acc' : epoch_val_accuracy,
                'validation_loss': epoch_val_loss,
                }, checkpoint_file)
    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

In [ ]:
print('output argmax:', output.argmax(dim=1))
print(len(output))
print(len(label))
print(label.shape, output.shape)
criterion(output, label)

In [ ]:
label

In [ ]:
# val_output = model(data)
# val_loss = criterion(val_output, label)

# acc = (val_output.argmax(dim=1) == label).float().mean()
# print('val_output:', val_output)
print('val_output argmax:', val_output.argmax(dim=1))
print(len(val_output))
print(len(label))
print(label.shape, val_output.shape)
print (data.shape)

In [ ]:
checkpoint = torch.load("/kaggle/input/model-weights/model_weights")
model.load_state_dict(checkpoint['model_state_dict'])
criterion.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch'] + 1

In [ ]:
from IPython.display import FileLink
FileLink(r'model_weights')

In [ ]:
from einops import rearrange, repeat
def get_latent(vit_model, img):
    x = vit_model.to_patch_embedding(img)
    b, n, _ = x.shape

    cls_tokens = repeat(vit_model.cls_token, '() n d -> b n d', b = b)
    x = torch.cat((cls_tokens, x), dim=1)
    x += vit_model.pos_embedding[:, :(n + 1)]
    x = vit_model.transformer(x)

    x = x.mean(dim = 1) if vit_model.pool == 'mean' else x[:, 0]

    return vit_model.to_latent(x)


In [ ]:
unshuffled_train_dataloader = DataLoader(train_dataset, batch_size=1)

In [ ]:
embeddings = np.zeros((len(train_dataset), 1000))
labels = []
model.eval()
imgs = []
for i, sample in enumerate(tqdm.tqdm(unshuffled_train_dataloader)):
#     if i >= 400:
#         break
    img, label = sample
#     print (label)
    labels.append(int(label))
    img = img.to(device)
    label = label.to(device)
#     img = img.unsqueeze(1)

#     embedding = get_latent(model, img)
    embedding = model(img)
    embeddings[i] = (embedding.cpu().detach().numpy())
    original_image = unnormalize_img(einops.rearrange(img.cpu().detach().numpy(), "b c w h -> w h (b c)"))
    imgs.append(original_image)
embeddings = np.array(embeddings)

In [ ]:

np.save("imgs.npy", imgs)

from IPython.display import FileLink
FileLink(r'imgs.npy')

In [ ]:
embeddings.shape

In [ ]:
from sklearn.manifold import TSNE

# The default of 1,000 iterations gives fine results, but I'm training for longer just to eke
# out some marginal improvements. NB: This takes almost an hour!
tsne = TSNE()

low_dim_embeddings = tsne.fit_transform(embeddings)
# low_dim_embeddings = embeddings

In [ ]:
cmap = {0: 'red', 1: 'blue'}
c = labels
plt.scatter(low_dim_embeddings[:, 0], low_dim_embeddings[:, 1], c=c, cmap = "Accent")
plt.colorbar()

In [ ]:
from numpy import random
from scipy.spatial import distance

def closest_node(node, nodes):
#     print (distance.cdist(node, nodes).shape)
    return np.argsort(distance.cdist(node, nodes))[:, :5][0]



In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

def image_grid(img_arr):
    fig = plt.figure(figsize=(20., 20.))
    grid = ImageGrid(fig, 111, 
                     nrows_ncols=(1, 5),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes
                     )

    for ax, im in zip(grid, img_arr):
        ax.imshow(im)

    plt.show()

In [ ]:
indices = closest_node(np.array([[40,40]]), low_dim_embeddings)
top_images = np.array(imgs)[indices]
print("NO FINDING IMAGES:")
print(np.array(labels)[indices])
image_grid(top_images)

In [ ]:
indices = closest_node(np.array([[60,-60]]), low_dim_embeddings)
top_images = np.array(imgs)[indices]
print("CARDIOMEGALY IMAGES:")
print(np.array(labels)[indices])
image_grid(top_images)

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax  = fig.add_subplot(111)
cmap = {0: 'red', 1: 'blue'}
c = labels
x = low_dim_embeddings[:, 0]
y = low_dim_embeddings[:, 1]
ax.scatter(low_dim_embeddings[:, 0], low_dim_embeddings[:, 1], c=c, cmap = "Accent")
# ax.colorbar()

def onclick(event):
    ix, iy = event.xdata, event.ydata
    print("I clicked at x={0:5.2f}, y={1:5.2f}".format(ix,iy))

    # Calculate, based on the axis extent, a reasonable distance 
    # from the actual point in which the click has to occur (in this case 5%)
    ax = plt.gca()
    dx = 0.05 * (ax.get_xlim()[1] - ax.get_xlim()[0])
    dy = 0.05 * (ax.get_ylim()[1] - ax.get_ylim()[0])

    # Check for every point if the click was close enough:
    for i in range(len(x)):
        if(x[i] > ix-dx and x[i] < ix+dx and y[i] > iy-dy and y[i] < iy+dy):
            plt.imshow(imgs[i])
            print("You clicked close enough!")

cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()